In [50]:
import numpy as np
import caffe
import cv2
from scipy.spatial import distance
from sklearn import svm
import cPickle as pickle
import os, sys
import subprocess
import lmdb

In [47]:
input_file = '/Users/ecsark/Documents/visualdb/project/artwork-explorer/data/input.txt'
model_def = '/Users/ecsark/Documents/visualdb/project/artwork-explorer/data/test.prototxt'
output_dir = '/Users/ecsark/Documents/visualdb/project/artwork-explorer/data/feature_output/'
extract_feature_bin = '/Users/ecsark/Documents/visualdb/caffe/distribute/bin/extract_features.bin'
model = '/Users/ecsark/Documents/visualdb/caffe/models/vgg/model.caffemodel'

feature_def = 'fc7'
run_command = [extract_feature_bin, model, model_def, feature_def, output_dir, '1', 'lmdb']

In [49]:
def read_from_db(db_dir):
    lmdb_env = lmdb.open(db_dir)
    lmdb_txn = lmdb_env.begin()
    lmdb_cursor = lmdb_txn.cursor()
    datum = caffe.proto.caffe_pb2.Datum()

    D = []

    for idx, (key, value) in enumerate(lmdb_cursor):
        datum.ParseFromString(value)
        data = caffe.io.datum_to_array(datum)
        D.append(data.flatten())

    lmdb_env.close()
    return D

In [52]:
subprocess.call(['rm', '-rf', output_dir])
ret = subprocess.call(run_command)
if ret == 0:
    ft = read_from_db(output_dir)[0]
    print ft
else:
    print "Feature extraction failed"
    


[ 1.92554736  2.2056694   0.         ...,  1.22003651  0.          0.        ]


In [41]:
def execute(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # Poll process for new output until finished
    while True:
        nextline = process.stdout.readline()
        if nextline == '' and process.poll() != None:
            break
        sys.stdout.write(nextline)
        sys.stdout.flush()

    print process.communicate()[0]
    print process.returncode

In [46]:
#call([extract_feature_bin, model, model_def, feature_def, output_dir, '1', 'lmdb'])
run_command = [extract_feature_bin, model, model_def, feature_def, output_dir, '1', 'lmdb']
execute(run_command)

E0406 14:41:58.797317 1980665856 extract_features.cpp:62] Using CPU
[libprotobuf WARNING google/protobuf/io/coded_stream.cc:537] Reading dangerously large protocol message.  If the message turns out to be larger than 2147483647 bytes, parsing will be halted for security reasons.  To increase the limit (or to disable these warnings), see CodedInputStream::SetTotalBytesLimit() in google/protobuf/io/coded_stream.h.
[libprotobuf WARNING google/protobuf/io/coded_stream.cc:78] The total number of bytes read was 574671595
E0406 14:42:01.041414 1980665856 extract_features.cpp:133] Extacting Features
E0406 14:42:01.787493 1980665856 extract_features.cpp:177] Extracted features of 1 query images for feature blob fc7
E0406 14:42:01.800042 1980665856 extract_features.cpp:182] Successfully extracted the features!

0


In [17]:
cmd = ' '.join([extract_feature_bin, model, model_def, feature_def, output_dir, '1', 'lmdb'])
p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
p.stdout.read()

'E0406 13:38:40.402632 1980665856 extract_features.cpp:62] Using CPU\n'